#### Dependencies

In [1]:
# System dependencies
from os import listdir
import time as clock
from IPython.display import clear_output

In [2]:
import pickle
import gzip
import re

In [3]:
# Data manipulation dependencies
import pandas as pd
import numpy as np
import datetime as dt

In [4]:
# Directory to save to
save_dir = '../data/continued/'

# Directory of prev job scans
id_dir = '../data/labels/IDs/'

In [5]:
# Directory of recent saved comet jobs
source_dir = '/oasis/projects/nsf/sys200/stats/xsede_stats/'

In [6]:
contents = listdir(source_dir)
contents

['archive_of_archive',
 'gordon_hostfile_logs',
 'gordon_pickles',
 'comet_accounting',
 'gordon_accounting',
 'comet_pickles',
 'archive',
 '.htaccess',
 'comet_hostfile_logs']

In [7]:
possible = [ source_dir+file_name for file_name in listdir(source_dir) ]

for item in possible:
    try:
        listdir(item)
    except:
        print(item)

/oasis/projects/nsf/sys200/stats/xsede_stats/gordon_hostfile_logs
/oasis/projects/nsf/sys200/stats/xsede_stats/gordon_pickles
/oasis/projects/nsf/sys200/stats/xsede_stats/gordon_accounting
/oasis/projects/nsf/sys200/stats/xsede_stats/comet_pickles
/oasis/projects/nsf/sys200/stats/xsede_stats/.htaccess
/oasis/projects/nsf/sys200/stats/xsede_stats/comet_hostfile_logs


In [8]:
locs = { 'aofa': source_dir+'archive_of_archive',
         'job_info': source_dir+'comet_accounting',
         'arc': source_dir+'archive'
         #'host_info': source_dir+'comet_hostfile_logs',
         #'old_pickles': source_dir+'comet_pickles'
       }

In [9]:
for key,loc in locs.items():
    print(loc)

/oasis/projects/nsf/sys200/stats/xsede_stats/archive_of_archive
/oasis/projects/nsf/sys200/stats/xsede_stats/comet_accounting
/oasis/projects/nsf/sys200/stats/xsede_stats/archive


## Prep Cleaning

In [30]:
def get_time( spec=None ):
    try:
        return clock.strftime("%Y/%m/%d %H:%M:%S", clock.localtime( spec ))
    except:
        return "0000/00/00 00:00:00"

In [11]:
def check_header( line ):
    if line.find(" ") < 0:
        try:
            return line[0] == '%'
        except:
            return False
        
    else:
        chunks = line.split(" ")
        try:
            return (chunks[0][0] == '%') or ( chunks[2].find("comet") >= 0 )
        except:
            return False

In [12]:
def check_job( chunk ):
    return chunk.find("-") == -1

In [13]:
def open_txt( txt_file ):
    
    with open( txt_file, "rt" ) as f:
        lines = f.readlines()
        f.close()
    
    return lines

In [14]:
def unzip_txt( gzipped ):
    
    with gzip.open( gzipped, 'rt') as f:
        lines = f.readlines()
        f.close()
    
    return lines

In [15]:
def quick_save( obj, label=get_time() ):
    
    try:
        out_file = open( label, 'wb')
        pickle.dump( obj, out_file)
        
        # double check save
        check_cpicore_set = pickle.load(open(cpiset_out, 'rb'))
        check_cpicore_set = None
        
    except:
        "There was a problem pickling the object - Save manually."

In [16]:
def info_dict( rules, info ):
    rules_list = rules.split("|")
    
    if len(rules_list) != len(info):
        return {}
    
    else:
        return { rules_list[i]:info[i] for i in range(len(rules_list)) }

In [57]:
def host_to_info_dict( zip_txt ):
    contents = unzip_txt( zip_txt )
    out_dict = { "Host": {} }
    host_info = { "Timestamp":{} }
    info_dict = { "Data":{},
                    "Job":"N/A",
                    "Schemas":{},
                    "Specs":[]
                }
    
    for line in contents:
            
        if line[0] == "$":
            info_dict["Specs"].append( format_spec( line ) )
            
        elif line[0] == "!":
            info_dict["Schemas"].update( format_schema( line ) )
        
        else:
            host_name = ""
            timestamp = ""
            
            if len(line) < 3 or check_header( line ):
                header_dict = format_header( line )
                
                if header_dict:
                    if check_job( header_dict["Jobid"] ):
                        info_dict["Job"] = { "Jobid": header_dict["Jobid"] }
                        
                    host_name = header_dict["Host"]
                    timestamp = header_dict["Timestamp"]
                    
                    time_data = { timestamp : {} }
                    out_dict[host_name] = time_data
                    
            else:        
                incoming = format_data( line )
                found_stat = incoming["Stat"]
                info_dict["Data"][ found_stat ] = incoming
                
    
    
                
    return out_dict

In [58]:
def format_header( line ):
    chunks = line.split(" ")
    
    try:
        if chunks[0][0] == '%':
            return {}
        else:
            return { "Timestamp": get_time( chunks[0] ), 
                     "Jobid": chunks[1],
                     "Host": chunks[2][:11] }
        
    except:
        print(line)
        return {}

In [59]:
def format_spec( line ):
    return line[1:-1]

In [60]:
def format_schema( line ):
    chunks = line.partition(" ")
    
    stat = chunks[0][1:]
    schemas = chunks[2:]
    
    return { stat:schemas }

In [65]:
def format_data( line ):
    chunks = line.split(" ")
    
    stat = chunks[0]
    dev = chunks[1]
    data = chunks[2:-1]
    
    return { "Stat": stat, "Device": dev, "Data": data }

### Parse file in archive

In [66]:
arc_data = [ locs['arc']+'/'+stamp for stamp in listdir(locs['arc']) ]

In [67]:
temp = listdir(arc_data[0])[0]
temp_loc = arc_data[0]+'/'+temp
temp_loc

'/oasis/projects/nsf/sys200/stats/xsede_stats/archive/comet-10-14.sdsc.edu/1587951665.gz'

In [68]:
host_to_info_dict( temp_loc )

{'Host': {},
 '': {'Timestamp': {},
  '': {'Data': {'Stat': 'vm',
    'Device': '-',
    'Data': ['3',
     '5914304241',
     '198784295',
     '0',
     '0',
     '97101369100',
     '98990571165',
     '961473995',
     '163956359',
     '185544931657',
     '4036335',
     '156603337',
     '0',
     '26253111',
     '15755',
     '0',
     '72880512',
     '0',
     '85196',
     '286182',
     '40',
     '27390',
     '325',
     '2',
     '1',
     '0']},
   'Job': {'Jobid': '32951517'},
   'Schemas': {'block': ('rd_ios,E rd_merges,E rd_sectors,E,U=512B rd_ticks,E,U=ms wr_ios,E wr_merges,E wr_sectors,E,U=512B wr_ticks,E,U=ms in_flight io_ticks,E,U=ms time_in_queue,E,U=ms\n',),
    'cpu': ('user,E,U=cs nice,E,U=cs system,E,U=cs idle,E,U=cs iowait,E,U=cs irq,E,U=cs softirq,E,U=cs\n',),
    'ib': ('excessive_buffer_overrun_errors,E,W=32 link_downed,E,W=32 link_error_recovery,E,W=32 local_link_integrity_errors,E,W=32 port_rcv_constraint_errors,E,W=32 port_rcv_data,E,U=4B,W=32 port_r

In [ ]:
info = 

In [ ]:
info

### Parse file in comet_accounting

In [ ]:
job_data = [ locs['job_info']+'/'+stamp for stamp in listdir(locs['job_info']) ]

In [ ]:
temp = job_data[0]
temp

In [ ]:
job_sample = open_txt( temp )

In [ ]:
rules = job_sample[0]
rules.split("|")

In [ ]:
job_sample[1].split("|")

In [ ]:
job_sample[1:]

In [ ]:
nodes_by_date = {}
unsaved = []

for date in job_data:
    try:
        
        # skip alt files
        #check_stamp = int( date[-14] )
        
        # read in file contents
        contents = open_txt( date )
    
        # formatting
        label = date[-14:-4]
        rules = contents[0]
        jobs = contents[1:]
        
        # template to save
        nodes_by_date[ label ] = {}
        nodes_by_date[ label ]["multiple"] = {}
        nodes_by_date[ label ]["rules"] = rules
        
        # run through lines in file
        for job in jobs:
            line = job.split("|")
            node = line[-1]
            info = info_dict( rules, line )
            
            # save multiple node jobs to specified loc
            if len(node) > 12:                
                nodes_by_date[ label ][ "multiple" ][ node ] = info
            
            else:
                nodes_by_date[ label ][ node[:11] ] = info
                
    except:
        unsaved.append(date)

In [ ]:
nodes_by_date['2019-09-19']

### Parse file in archive of archive

In [ ]:
aofa_data = [ locs['aofa']+'/'+stamp for stamp in listdir(locs['aofa']) ]

In [ ]:
temp = listdir(aofa_data[0])[0]
aofa_data[0]+'/'+temp

In [ ]:
aofa_sample = unzip_txt( aofa_data[0]+'/'+temp )

In [ ]:
aofa_sample

# PREVIOUS PROCEDURE

In [ ]:
t0 = clock.time()
n = len(files)
valid = []
invalid = []
scanned = 0

for file_name in files[:10]:
    scanned += 1
    perc_scanned = np.round( scanned / n * 100, 2)
        
    clear_output(wait=True)
    print "Processing file {} of {} files \t ({}% of total files)".format( scanned, n, perc_scanned )
        
    try:
        check_file = open( source_dir+file_name, 'r' )
        
        for line in check_file:
            print(line)
        
        check_file.close()
    except:
        next
#        job_obj = pickle.load( pickle_file )
#        
#        # Save data
#        valid.append(job_obj)
#        summary[date]["Saved"] += 1
#        
#        pickle_file.close()
#            
#    except:
#        invalid.append( file_name )
#            
#    print
#    print "Run time: {}s".format( np.round( clock.time() - t0, 1 ) )

In [ ]:
len(invalid) == len(files)


# Resume

In [ ]:
# List of date directories in source_dir
dates_list = [ date for date in listdir(source_dir) if len(listdir(source_dir+date)) > 0 ]
all_files = [ source_dir+date+'/'+file_name for date in listdir(source_dir) for file_name in listdir(source_dir+date) ]

#### Confirm Already Scanned Jobs

In [ ]:
collected = [ file_name[12:19] for file_name in listdir('../data/raw') ]
remaining = [ file_name for file_name in all_files if file_name[72:] not in collected ]        

In [ ]:
print "Total files:\t", len(all_files)
print "Collected:\t", len(collected)
print "--------------------------"
print "Remaining:\t", len(remaining)

## Prep Data Cleaning

In [ ]:
def check_val( val ):
    try:
        val = float(val)
        return val
    except:
        return 0
    else:
        return 0
        
def convert_dt( val ):
    return dt.datetime.utcfromtimestamp( val ).strftime( "%Y-%m-%d %H:%M:%S" )

def get_schemas( job ):
    return { stat:schema.keys() for stat,schema in job.schemas.items() }

def get_indices( job, host ):
    indices = []
    stats = [ stat for stat in job.schemas.keys() if stat in host.stats.keys() ]
    schemas = { stat:schema.keys() for stat,schema in job.schemas.items() }
    cores = { stat:core.keys() for stat,core in host.stats.items() }
    
    for stat in stats:
        for core in cores[stat]:
            for schema in schemas[stat]:
                indices.append( (stat,core,schema) )
             
    return indices

def get_times( job, host ):
    times = [ job.start_time ]
    times.extend( host.times )
    times.append( job.end_time )
    return [ convert_dt(t) for t in times ]

def clean_list( data_list ):
    return [ check_val( x ) for x in data_list ]
    
def get_data( host, row_labels ):
    data = { label:[] for label in row_labels }
    
    for stat,node in host.stats.items():
        for core,matrix in node.items():
            matrix = matrix.T
            for i in range(len(matrix)):
                data[stat,core] = clean_list( matrix[i] )
    return data

def fill_df( template_df, data_dict):
    for row,data in data_dict.items():
        template_df.loc[row].update( pd.Series(data) )
    return df

In [ ]:
def cpicore ( job_df ):
    data = job_df.loc['intel_hsw']
    times = job_df.columns.tolist()
    cpicore_list = []
    
    for i in range(1, len(times)):
        chunk = data[times[:i+1]]
        devices = { row : np.mean(col.values) for row,col in chunk.iterrows() }
        avg_c = { key[0]:0 for key,val in devices.items() }
        sum_avgs = 0
        
        for key,val in avg_c.items():
            avg_c[ key ] = devices[ (key, 'CLOCKS_UNHALTED_CORE') ] / devices[ (key, 'INSTRUCTIONS_RETIRED') ]
    
        for key,val in avg_c.items():
            sum_avgs += val
            
        cpicore_list.append(sum_avgs/24)
    
    return cpicore_list

In [ ]:
#def find_next( current, unsorted ):
#    target = current + 00:10:00
#    found = unsorted[0]
#    proximity = target - found
#    
#    if len(unsorted) > 1:
#        for i in range(len(unsorted)):
#            if target - unsorted[i] < proximity:
#                found = unsorted[i]
#                proximity = target - found
#    return found
#
#def fill_sorted( start, unsorted ):
#    sorted_list = []
#    
#    for i in range(len(unsorted)):
#        current = sorted_list[i]
#        next_time = find_next( current, unsorted )
#        sorted_list[i+1] = next_time
#        
#def sort_times( job ):
#    start = job.start
#    mid = job.times
#    end = job.end
#    
#    if start == end:
#        return [start]
#    elif len(mid) < 1:
#        return [start, end]
#    elif len(mid) < 2:
#        return [start, mid[0], end]
#    else:
#        return fill_sorted( start, mid.append(end) )

# Access and open pickled job files
**Process:**
    - Iterate through the non-empty date folders available in source_dir
    - A file is saved in valid_jobs if:
        * The pickled file is a Job object
    - Exceptions are skipped

In [ ]:
cut = remaining
n = len(cut)

In [ ]:
summary = {}

for file_name in cut:
    date = file_name[61:71]
    
    if date in summary:
        summary[date]["Total"] += 1
    else:
        summary[date] = {}
        summary[date]["Total"] = 1
        summary[date]["Saved"] = 0

In [ ]:
summary

#### Catch invalid files

In [ ]:
t0 = clock.time()
valid = []
invalid = []
scanned = 0

for file_name in cut:
    date = file_name[61:71]
    scanned += 1
    perc_scanned = np.round( scanned / n * 100, 2)
        
    clear_output(wait=True)
    print "Processing file {} of {} files \t ({}% of total files)".format( scanned, n, perc_scanned )
        
    try:
        pickle_file = open( file_name, 'rb' )
        job_obj = pickle.load( pickle_file )
        
        # Save data
        valid.append(job_obj)
        summary[date]["Saved"] += 1
        
        pickle_file.close()
            
    except:
        invalid.append( file_name )
            
    print
    print "Run time: {}s".format( np.round( clock.time() - t0, 1 ) )

In [ ]:
invalid_out = '../src/data/summary_stats/raw_metrics/invalid.pkl'
out_file = open(invalid_out, 'wb')
pickle.dump(invalid, out_file)
#invalid = pickle.load(open(invalid_out, 'rb'))

In [ ]:
valid_out = '../src/data/summary_stats/raw_metrics/all_rows.pkl'
out_file = open(all_rows_out, 'wb')
pickle.dump(all_rows, out_file)
#valid = pickle.load(open(valid_out, 'rb'))

In [ ]:
len(invalid)

In [ ]:
len(valid)

In [ ]:
print "Total job objects collected:\t", len(valid)
print
print "\tBreakdown of files"
print "========================="

for date,info in summary:
    
    print "Date:\t", date
    print info["Saved"], "files saved out of", info["Total"]
    print

# Begin Processing

In [ ]:
job_objects = valid
m = float(sum([len(job.hosts.keys()) for job in job_objects]))

In [ ]:
cut = len(job_objects)/2
first = 0
stop = len(job_objects)
rem = stop - first

print "Total Jobs (this date):\t\t", len(job_objects)
print "Total Host,Job Pairs:\t\t", int(m)
print("------------------------------------")
print "Remaining jobs to scan:\t\t", int(rem)

In [ ]:
job_dfs = {}
t0 = clock.time()
total = 0
current = 0

for job_idx in range( first, stop ):
    job = job_objects[ job_idx ]
    schemas = get_schemas( job )
    total += 1
    
    # support for tracking progress in below print statements
    clear_output(wait=True)
    
    # iterate through each host object job was run on
    for host_name, host in job.hosts.iteritems():
        print("Processing hosts for job {} of {} \t ({}% of total)".format(job_idx+1, stop, np.round( (current+first)/m*100, 2)))
        current += 1
        
        # build MultiIndex for df 
        idx_labels = get_indices( job, host )
        indices = pd.MultiIndex.from_tuples( idx_labels, names=['Stat', 'Device', 'Schema'] )
                    
        # process timestamps
        times = get_times( job, host )
    
        # collect job data
        data = get_data( host, idx_labels )
        
        # create df with MultiIndex, ordered times
        df = pd.DataFrame( index=indices, columns=times ).sort_index()
        
        # fill df
        for stat,devices in host.stats.items():
            for device,data_matrix in devices.items():
                for t_idx in range( len(data_matrix) ):
                    timestamp = times[t_idx]
                    
                    for metric_idx in range( len(data_matrix[ t_idx ]) ):
                        metric = schemas[stat][metric_idx]
                        row_label = (stat,device,metric)
                        datum = data_matrix[t_idx][metric_idx]
                        
                        df.loc[row_label][timestamp] = check_val(datum)
        
        # save job info from DataFrame to csv file
        df.to_csv( path_or_buf=save_dir+"{}_{}.csv".format( host_name, job.id ) )

In [ ]:
# check that no job was missed
if total == ( stop-first ):
    print "Success!"
else:
    print stop - first - total, "jobs missing"